In [58]:
import pandas as pd
import numpy as np
####

Country = "Nigeria"
Country_abbr = "Nig" # Pol, Hun
text_name = "4625184453"+".txt"

####


df = pd.read_csv("../data/" + Country + '_DataFrame.csv')
''' Select data frame that ends at 2020-03 '''
end_index = None
for index in range(len(df)):
    if df.iloc[index, 0] == "2020-03":
        end_index = index
        break
assert (end_index is not None)


''' Scale the data '''
def scale_series(series, mean, std):
    return (series - mean)/std
for col_num in range(1, len(df.columns)):
    df.iloc[:, col_num] = scale_series(df.iloc[:, col_num], df.iloc[:end_index+1, col_num ].mean(), df.iloc[:end_index+1, col_num ].std())


In [59]:

file_path = "./Output/" + Country_abbr +"/corr/"


dict_weight = dict()

with open(file_path+text_name, "r") as f:
    is_Weight = False
    line = f.readline()
    while len(line.strip()):
        if line.strip().startswith("_") or line.strip().startswith("-"):
            line = f.readline()
            continue
        if is_Weight:
            
            name, weight = line.strip().split(":")
            name = name.strip()
            weight = weight.strip()
            weight = float(weight)
            dict_weight[name] = weight
        else:
            if line.startswith("Weight"):
                is_Weight = True
        line = f.readline()


In [60]:
df

,Date,Term_Premium,Risk_Premium,Stock_Prices_mom%_change,Stock_Prices_mom24mma%_change,Stock_Market_Volatility,REER,Current_Account_Balance_change_yryr%,Current_Account_Balance_over_GDP,One-Day_Repo_Rate_AVG,One-Day_Repo_Rate_EOP,Policy_Rate_&_Fed_Funds_Rate_Differential_AVG,Policy_Rate_&_Fed_Funds_Rate_Differential_EOP,Broad_Money_mo12m%_change,Velocity_of_Money_mo12m%_change,Portfolio_Flows,Foreign_Exchange_Reserve_change_yryr%,Foreign_Exchange_Reserve_over_GDP
0,1995-01,NaN,NaN,NaN,NaN,NaN,0.363918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.772846,NaN
1,1995-02,NaN,NaN,NaN,NaN,NaN,0.462917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.816730,NaN
2,1995-03,NaN,NaN,NaN,NaN,NaN,0.453880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.557298,NaN
3,1995-04,NaN,NaN,NaN,NaN,NaN,0.581097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.949300,NaN
4,1995-05,NaN,NaN,NaN,NaN,NaN,0.731815,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.664757,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,2023-05,-1.729959,-0.064761,0.822095,0.353248,-0.222185,0.569456,NaN,NaN,2.653089,2.800533,0.970391,1.096901,NaN,NaN,-0.394829,-0.528994,-0.239614
341,2023-06,-2.008310,0.065654,1.233755,0.653694,-2.046732,-0.062097,NaN,NaN,2.807038,2.800533,1.105975,1.096901,NaN,NaN,-0.518307,-0.580840,0.159660
342,2023-07,-0.273254,0.583884,0.694806,0.798894,-3.160313,-0.013186,NaN,NaN,2.828591,2.895971,1.040895,1.096901,NaN,NaN,-0.518307,NaN,NaN
343,2023-08,-0.232894,0.158318,0.398229,0.851313,-0.741509,0.076652,NaN,NaN,2.902486,2.895971,1.105975,1.096901,NaN,NaN,-0.518307,NaN,NaN


In [61]:

df = df[["Date",*dict_weight.keys()]].dropna().reset_index()


In [62]:
df

,index,Date,Term_Premium,Stock_Prices_mom24mma%_change,Policy_Rate_&_Fed_Funds_Rate_Differential_AVG,Foreign_Exchange_Reserve_change_yryr%
0,157,2008-02,-0.327069,1.829147,-1.224216,0.242107
1,158,2008-03,-0.329389,1.453202,-1.056092,0.352948
2,159,2008-04,-0.443049,0.977077,-0.840894,0.345115
3,160,2008-05,-0.227327,0.786136,-0.709758,0.299910
4,161,2008-06,-1.177896,0.438384,-0.793259,0.329104
...,...,...,...,...,...,...
180,337,2023-02,-1.013205,0.516664,0.941214,-0.498119
181,338,2023-03,-1.039648,0.388805,0.913446,-0.522163
182,339,2023-04,-0.910679,0.158248,1.021913,-0.501060
183,340,2023-05,-1.729959,0.353248,0.970391,-0.528994


In [63]:

FCI = np.matmul(df[dict_weight.keys()],np.square(list(dict_weight.values())))
Date = df["Date"]
df_output = pd.DataFrame({"Date":Date, "FCI":FCI})

end_index = None
for index in range(len(df)):
    if df_output.iloc[index, 0] == "2020-03":
        end_index = index
        break
assert (end_index is not None)
for col_num in range(1, len(df_output.columns)):
    df_output.iloc[:, col_num] = scale_series(df_output.iloc[:, col_num], df_output.iloc[:end_index+1, col_num ].mean(), df_output.iloc[:end_index+1, col_num ].std())

df_output.to_csv("Output/" + Country_abbr+"/FCI.csv", index = False)
df_output.to_csv("PCA results for visualization/"+ Country + "/FCI.csv", index = False)
